## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import widgetsnbextension
import gmaps.datasets
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import ipywidgets as widgets

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,57.88,90,75,8.99,heavy intensity rain
1,1,Arlit,NE,18.7369,7.3853,97.20,21,100,11.25,overcast clouds
2,2,Kaitangata,NZ,-46.2817,169.8464,37.58,85,97,9.08,overcast clouds
3,3,Nioro,GM,13.3500,-15.7500,77.61,87,100,7.02,overcast clouds
4,4,Varkaus,FI,62.3153,27.8730,64.51,97,100,4.54,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                       
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,57.88,90,75,8.99,heavy intensity rain
1,1,Arlit,NE,18.7369,7.3853,97.20,21,100,11.25,overcast clouds
2,2,Kaitangata,NZ,-46.2817,169.8464,37.58,85,97,9.08,overcast clouds
3,3,Nioro,GM,13.3500,-15.7500,77.61,87,100,7.02,overcast clouds
4,4,Varkaus,FI,62.3153,27.8730,64.51,97,100,4.54,overcast clouds
5,5,Port Elizabeth,ZA,-33.9180,25.5701,58.10,81,100,8.46,overcast clouds
6,6,Akyab,MM,20.1500,92.9000,80.80,86,100,2.66,overcast clouds
7,7,Barrow,US,71.2906,-156.7887,34.45,91,100,9.86,overcast clouds
8,8,Katsuura,JP,35.1333,140.3000,84.31,88,100,23.13,overcast clouds
9,9,Sao Sebastiao,BR,-23.7600,-45.4097,69.39,73,2,1.86,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                675
City                   675
Country                673
Lat                    675
Lng                    675
Max Temp               675
Humidity               675
Cloudiness             675
Wind Speed             675
Current Description    675
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID                673
City                   673
Country                673
Lat                    673
Lng                    673
Max Temp               673
Humidity               673
Cloudiness             673
Wind Speed             673
Current Description    673
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,57.88,heavy intensity rain,-33.9258,18.4232,
1,Arlit,NE,97.20,overcast clouds,18.7369,7.3853,
2,Kaitangata,NZ,37.58,overcast clouds,-46.2817,169.8464,
3,Nioro,GM,77.61,overcast clouds,13.3500,-15.7500,
4,Varkaus,FI,64.51,overcast clouds,62.3153,27.8730,
5,Port Elizabeth,ZA,58.10,overcast clouds,-33.9180,25.5701,
6,Akyab,MM,80.80,overcast clouds,20.1500,92.9000,
7,Barrow,US,34.45,overcast clouds,71.2906,-156.7887,
8,Katsuura,JP,84.31,overcast clouds,35.1333,140.3000,
9,Sao Sebastiao,BR,69.39,clear sky,-23.7600,-45.4097,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", float("NaN"), inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,57.88,heavy intensity rain,-33.9258,18.4232,Southern Sun Waterfront Cape Town
1,Arlit,NE,97.20,overcast clouds,18.7369,7.3853,Case de Passage De la Prefecture
2,Kaitangata,NZ,37.58,overcast clouds,-46.2817,169.8464,Kaitangata Motor Camp
4,Varkaus,FI,64.51,overcast clouds,62.3153,27.8730,Hotel Oscar
5,Port Elizabeth,ZA,58.10,overcast clouds,-33.9180,25.5701,39 On Nile Guest House
6,Akyab,MM,80.80,overcast clouds,20.1500,92.9000,Hotel Memory
7,Barrow,US,34.45,overcast clouds,71.2906,-156.7887,King Eider Inn
8,Katsuura,JP,84.31,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
9,Sao Sebastiao,BR,69.39,clear sky,-23.7600,-45.4097,Hotel Recanto dos Pássaros
10,Sitka,US,60.69,overcast clouds,57.0531,-135.3300,Westmark Sitka


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
Hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=Hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))